Being an ardent cricket fun it gave me immense pleasure to analyse the data. I tried to analyze the data from various aspects and tried to represent it through tables and graphs. Hope this helps someone to gain better inside or take it to next level.

This notebook contains:
1. Team Analysis
2. Stadium Analysis
3. Player Analysis 
4. Match Analysis etc.

The libraries used are:
1. pandas
2. plotly
3. numpy
4. matplotlib


In [1]:
# Library Initializations
import pandas as pd
import plotly.graph_objs as go
import numpy as np
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.tools import FigureFactory as FF

init_notebook_mode(connected=True)
pd.set_option('display.max_columns', 40)

# Read data from files
# The file is stored at the same location as the notebook
matches = pd.read_csv("matches.csv")
deliveries = pd.read_csv("deliveries.csv")



### Sneak Peak of the data

In [2]:
matches.head(2)

,id,season,city,date,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match,venue,umpire1,umpire2,umpire3
0,1,2017,Hyderabad,2017-04-05,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,0,Sunrisers Hyderabad,35,0,Yuvraj Singh,"Rajiv Gandhi International Stadium, Uppal",AY Dandekar,NJ Llong,NaN
1,2,2017,Pune,2017-04-06,Mumbai Indians,Rising Pune Supergiant,Rising Pune Supergiant,field,normal,0,Rising Pune Supergiant,0,7,SPD Smith,Maharashtra Cricket Association Stadium,A Nand Kishore,S Ravi,NaN


In [3]:
deliveries.head(2)

,match_id,inning,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,is_super_over,wide_runs,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,player_dismissed,dismissal_kind,fielder
0,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,1,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN
1,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,2,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN


### Number of wins by a team per season

In [5]:
no_of_wins = pd.DataFrame(matches.groupby(['winner','season']).id.count()).reset_index()
# No of Wins by Chennai Super Kings
no_of_wins

data = [go.Bar(
            x=no_of_wins[no_of_wins.winner=='Chennai Super Kings']['season'],
            y=no_of_wins[no_of_wins.winner=='Chennai Super Kings']['id'],
            marker={
                'color': no_of_wins[no_of_wins.winner=='Chennai Super Kings']['id'],
                'colorscale': 'Viridis'
            }
    )]

layout=go.Layout(
        title="No of wins"
       )
 
updatemenus=list([
    dict(
        buttons=list([   
            dict(
                args=['type', 'surface'],
                label='3D Surface',
                method='restyle'
            ),
            dict(
                args=['type', 'heatmap'],
                label='Heatmap',
                method='restyle'
            )             
        ]),
        direction = 'down',
        pad = {'r': 10, 't': 10},
        showactive = True,
        x = 0.1,
        xanchor = 'left',
        y = 1.1,
        yanchor = 'top' 
    ),
])

annotations = list([
    dict(text='Choose team:', x=0, y=1.085, yref='paper', align='left', showarrow=False)
])
layout['updatemenus'] = updatemenus
layout['annotations'] = annotations


fig = go.Figure(data=data, layout=layout)

iplot(fig, filename='group-bar')


In [6]:
layout = go.Layout(
    title='No of Wins'
)



## Matches per ground

In [7]:
venue_list = matches.groupby('venue').venue.count()

data = [go.Bar(
            y=pd.DataFrame(venue_list.index)['venue'],
            x=pd.DataFrame(venue_list.values)[0],
            orientation = 'h',
            marker={
                'color': pd.DataFrame(venue_list.values)[0],
                'colorscale': 'Jet'
            }
    )]
iplot(data, filename='basic-bar')

## Top 10 batsman with number of fours and sixes

The top batsman was selected based on total runs and then the number of 4s and 6s hit by them

In [8]:
df = pd.merge(matches, deliveries,how='inner',left_on='id',right_on='match_id')
top_10_batsman = df.groupby(['batsman']).batsman_runs.sum().sort_values(ascending=False)[0:10].index

no_of_fours = pd.DataFrame(df[df.batsman.isin(top_10_batsman) & (df.batsman_runs==4)].groupby(['batsman']).batsman_runs.count())
no_of_sixes = pd.DataFrame(df[df.batsman.isin(top_10_batsman) & (df.batsman_runs==6)].groupby(['batsman']).batsman_runs.count())


no_of_fours.reset_index(inplace=True)
no_of_sixes.reset_index(inplace=True)

no_of_fours.columns = ['batsman','no of fours']
no_of_sixes.columns = ['batsman','no of sixes']

total_no_fours_sixes = no_of_fours.merge(no_of_sixes, left_on='batsman',right_on='batsman')

trace1 = go.Bar(
            x=total_no_fours_sixes['batsman'],
            y=total_no_fours_sixes['no of fours'],
            text=total_no_fours_sixes['no of fours'],
            textposition='auto',
            name = 'No of 4s'
        )
trace2= go.Bar(        
            x=total_no_fours_sixes['batsman'],
            y=total_no_fours_sixes['no of sixes'],
            text=total_no_fours_sixes['no of sixes'],
            textposition='auto',
            name = 'No of 6s'
        )

data = [trace1,trace2]
layout = go.Layout(
    barmode='group'
)

fig = go.Figure(data=data, layout=layout)

iplot(fig, filename='group-bar')

###  Closest matches
A Closest match is defined as 
1. The side batting first won by < 5 runs
2. The side chasing won with <= 2 wickets remaining

In [9]:
from IPython.core.display import HTML


winner_by_shortest_margin = pd.DataFrame(matches[(matches.win_by_runs<5) & (matches.win_by_wickets==0)].groupby(['season','winner']).id.count()).reset_index()
winner_by_shortest_margin.columns=['season','winner','no of times']

display(HTML(winner_by_shortest_margin.to_html()))
 
winner_by_least_wickets = pd.DataFrame(matches[(matches.win_by_runs==0) & (matches.win_by_wickets<=2)].groupby(['season','winner']).id.count()).reset_index()
winner_by_least_wickets.columns=['season','winner','no of times']

display(HTML(winner_by_least_wickets.to_html()))


,season,winner,no of times
0,2008,Chennai Super Kings,1
1,2008,Kings XI Punjab,1
2,2008,Royal Challengers Bangalore,1
3,2009,Kings XI Punjab,2
4,2009,Rajasthan Royals,2
5,2010,Kings XI Punjab,1
6,2010,Mumbai Indians,1
7,2010,Rajasthan Royals,1
8,2011,Chennai Super Kings,1
9,2012,Delhi Daredevils,1


,season,winner,no of times
0,2009,Rajasthan Royals,1
1,2009,Royal Challengers Bangalore,1
2,2010,Kings XI Punjab,1
3,2012,Mumbai Indians,1
4,2013,Royal Challengers Bangalore,1
5,2013,Sunrisers Hyderabad,1
6,2014,Rajasthan Royals,1
7,2015,Kings XI Punjab,1
8,2015,Kolkata Knight Riders,1
9,2016,Kolkata Knight Riders,1


### Number of wins in each city in every release

In [10]:
start_season, end_season = 2008, 2018
while start_season < end_season:
    wins_percity = pd.DataFrame(matches[matches['season'] == start_season].groupby(['winner', 'city'])['id'].count().unstack()).reset_index()
    wins_percity.fillna(value=0,inplace=True)
    wins_percity.dropna(axis='columns',how='all',inplace=True)
    wins_percity.dropna(inplace=True)

    city_in_season = matches[matches.season == start_season].city.unique()
    data = list();
    
    for each_city in city_in_season:
        trace = go.Bar(
            x=wins_percity['winner'],
            y=wins_percity[each_city],
            text=wins_percity[each_city],
            textposition='auto',
            name=each_city,
        )
        data.append(trace)
    layout = go.Layout(
        barmode='stack',
        title='Wins Per City in Season '+str(start_season),
    )
    fig = go.Figure(data=data, layout=layout)
    iplot(fig, filename='stacked-bar')
    start_season += 1

ValueError: cannot label index with a null key

### Impact of toss on the match outcome

In [ ]:
#Toss won vs Match won
win_count = pd.DataFrame(matches[matches.toss_winner == matches.winner].groupby('season').winner.count()).reset_index()
win_count.columns = ['season','both_toss_n_match_winner']

total_no_of_matches = pd.DataFrame(matches.groupby('season').id.count()).reset_index()
total_no_of_matches.columns = ['season','no_of_matches']

percentage_wins = win_count.merge(total_no_of_matches)
percentage_wins['percentage_wins'] = round(percentage_wins['both_toss_n_match_winner']*100/percentage_wins['no_of_matches'],2)

percentage_wins.set_index('season',inplace=True)

percentage_wins.plot.line()

### Bowler's Performance w.r.t. wickets

In [11]:
type_of_dismissal = ['caught','bowled', 'lbw', 'caught and bowled','stumped', 'hit wicket']
valid_wickets_df = pd.DataFrame(df[df.dismissal_kind.isin(type_of_dismissal)].groupby(['season','bowler']).inning.count())

top_5_wicket_takers = pd.DataFrame(valid_wickets_df.sort_values(by=["season","inning"],ascending=False).groupby('season').head(5)).reset_index()
top_5_wicket_takers

,season,bowler,inning
0,2017,B Kumar,26
1,2017,JD Unadkat,24
2,2017,JJ Bumrah,20
3,2017,MJ McClenaghan,19
4,2017,Imran Tahir,18
5,2016,B Kumar,23
6,2016,YS Chahal,21
7,2016,SR Watson,20
8,2016,DS Kulkarni,18
9,2016,DJ Bravo,17


### Number of deliveries bowled in superover in each season

In [12]:
balls_per_season = pd.DataFrame(df[df.is_super_over == 1].groupby('season').id.count()).reset_index()
balls_per_season.columns = ['season','no_of_superover_balls']
data = [go.Bar(
            x=balls_per_season['no_of_superover_balls'],
            y=balls_per_season['season'],
            text=balls_per_season['no_of_superover_balls'],
            textposition='auto',
            orientation = 'h',
            marker={
                'color': balls_per_season['season'],
                'colorscale': 'Jet'
            }
    )]
layout = go.Layout(
        title='No of balls bowled in SuperOver '
    )
fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='basic-bar')

In [15]:
a="ply"
a[0]='1'
a

TypeError: 'str' object does not support item assignment